# Cricket Ball Detection – Model Training 
This notebook covers dataset setup, preprocessing, verification, and YOLO model training using Colab GPU


 installs OpenCV, required for video processing.

In [ ]:
!pip install opencv-python

This cell mounts Google Drive to access dataset and training files.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.getcwd()

'/content'

This cell installs Roboflow and loads a dataset project. 
I have annotated 160+ images manually (using Roboflow interface) divided them into train, test, and validation sets, for training

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="###############") #not disclosed for privacy reason
project = rf.workspace("rasmalai-o8kwv").project("rasgulla-mjugh")
version = project.version(2)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 141.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Rasgulla-2 in yolov8:: 100%|██████████| 238/238 [00:00<00:00, 4524.44it/s]


In [ ]:
import os
os.chdir("drive/MyDrive/edgefleet assignment (1)/cricket-ball-tracker (1)")

In [ ]:
os.getcwd()

'/content/drive/MyDrive/edgefleet assignment (1)/cricket-ball-tracker (1)'

This cell lists files to verify dataset structure.

In [ ]:
import cv2
import pandas as pd
df_processed = pd.DataFrame()
for i in range(1,15):
    video_path = f"data (1)/processed_videos/{i}.mp4"

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        exit()

    # Fetch properties
    width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps    = cap.get(cv2.CAP_PROP_FPS)
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = frames / fps if fps > 0 else 0
    new_row = pd.DataFrame({"width":width, "height":height, "fps":fps, "frame_count":frames, "duration_sec":duration}, index=[0])
    df_processed = pd.concat([df_processed, new_row], ignore_index=True)

In [ ]:
df_processed

,width,height,fps,frame_count,duration_sec
0,1920.0,1080.0,40.0,31.0,0.775
1,1920.0,1080.0,40.0,293.0,7.325
2,1920.0,1080.0,40.0,262.0,6.550
3,1920.0,1080.0,40.0,265.0,6.625
4,1920.0,1080.0,40.0,243.0,6.075
5,1920.0,1080.0,40.0,361.0,9.025
6,1920.0,1080.0,40.0,237.0,5.925
7,1920.0,1080.0,40.0,209.0,5.225
8,1920.0,1080.0,40.0,274.0,6.850
9,1920.0,1080.0,40.0,201.0,5.025


This cell installs the Ultralytics YOLO package.

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.0 MB/s eta 0:00:00


This create a function to verify dataset structure.

In [ ]:
import os

def check_mismatch(img_dir, label_dir):
    imgs = sorted([f for f in os.listdir(img_dir) if f.endswith((".jpg", ".png"))])
    labels = sorted([f for f in os.listdir(label_dir) if f.endswith(".txt")])

    img_names = set([os.path.splitext(f)[0].lower() for f in imgs])
    label_names = set([os.path.splitext(f)[0].lower() for f in labels])

    missing_labels = img_names - label_names
    missing_images = label_names - img_names

    print("\n=== Missing Label Files ===")
    for x in missing_labels:
        print(x)

    print("\n=== Missing Image Files ===")
    for x in missing_images:
        print(x)


In [ ]:
import os
os.getcwd()

'/content/drive/MyDrive/edgefleet assignment (1)/cricket-ball-tracker (1)'

In [ ]:
import os
os.chdir("..")

In [ ]:
os.getcwd()

'/content/drive/MyDrive/edgefleet assignment (1)'

In [ ]:
print("Exists on disk:", os.path.exists("cricket-ball-tracker (1)/Rasgulla-2"))

Exists on disk: True


This cell lists files to verify dataset structure.

In [ ]:

# Run for train set
check_mismatch("cricket-ball-tracker (1)/Rasgulla-2/train/images", "cricket-ball-tracker (1)/Rasgulla-2/train/labels")

# Run for valid set
check_mismatch("cricket-ball-tracker (1)/Rasgulla-2/valid/images", "cricket-ball-tracker (1)/Rasgulla-2/valid/labels")


=== Missing Label Files ===

=== Missing Image Files ===

=== Missing Label Files ===

=== Missing Image Files ===


In [ ]:
os.getcwd()

'/content/drive/MyDrive/edgefleet assignment (1)'

In [ ]:
os.chdir("cricket-ball-tracker (1)/Rasgulla-2")

In [ ]:
os.getcwd()

'/content/drive/MyDrive/edgefleet assignment (1)/cricket-ball-tracker (1)/Rasgulla-2'

# Model Training

This installs TensorBoard for logging training metrics.

In [ ]:
!pip install tensorboard

This clears CUDA memory.

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


In [ ]:
from ultralytics.utils import SETTINGS

SETTINGS.update({"tensorboard": True})


This cell initializes a YOLO model and starts training using the provided dataset.

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt")   # medium model, T4-friendly

model.train(
    data="data.yaml",
    epochs=120,
    imgsz=1600,        # reduced  from 1920
    batch=4,          # reduced from 8
    device=0,
    amp=True,         # FP16 - reduces memory
    workers=2
)


Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=120, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1600, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ac87a781df0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

This cell copies training results back to Google Drive.

In [ ]:
!cp -r runs/detect/train6  /content/drive/MyDrive/
